In [237]:
import gurobipy as gb
import pandas as pd

import numpy as np
np.set_printoptions(linewidth=100)

import random
import os

import pprint
pp = pprint.PrettyPrinter(compact=True)

In [238]:
class Generator:
    def __init__(self, stops_num, peak_period, passengers_num, bus_trips,\
                tt_disutility, bus_capacity, tt_adj_stops, dwell_time_stop_i, tt_elasticity):
        self.S = stops_num
        self.T = peak_period
        self.P = passengers_num   # (expected demand p^k ?)
        self.B = bus_trips        # (constant frequency f0 ?)

        self.m = tt_disutility
        self.C = bus_capacity
        self.ta = tt_adj_stops
        self.d = dwell_time_stop_i
        self.e = tt_elasticity
        
        self.s = list(range(1,self.S+1))

        self.t = None
        self.c = None
        self.a = None
        self.K = None
        
        self.K_adj = None
        self.K_num = None
        
        self.gamma_k = None
        
        self.t_dict = None
        self.c_dict = None
        self.a_dict = None
        self.gamma_k_dict = None
        
        self.M_dict = None

        self.__start()
        
    def __start(self):
        self.K, self.K_adj, self.K_num, self.t, self.c, self.a, self.gamma_k = self.__gen_multi_matrix()
        self.t_dict, self.c_dict, self.a_dict, self.gamma_k_dict = self.__gen_multi_dict()
        self.M_dict = self.__build_dataframe()

    def __gen_multi_matrix(self):
        K, K_adj, K_num, t, c, a, gamma_k = [], [], [], [], [], [], []
        # K, K_adj, t, c
        for i in range(1,self.S):
            for j in range(i+1, self.S+1):
                K.append((i,j))
                if(j == i+1):
                    K_adj.append((i,j))
                    t.append(self.ta)
                    c.append(0.0)
                else: 
                    t.append(self.ta * (j-i) + self.d * ((j-1)-i))
                    c.append(self.d * ((j-1)-i))
        # a
        for i in t:
            a.append(np.divide(-self.e, i))
        # K_num
        K_num = list(range(len(K)))
        # gamma_k
        for s,d in K:
            gamma_k.append([(i,j) \
                            for i in self.s \
                            for j in self.s \
                            if(i>=s and j>=s and i<j and i<=d and j<=d)])
        return [K, K_adj, K_num, t, c, a, gamma_k]
    
    def __gen_multi_dict(self):
        t_dict, c_dict, a_dict, gamma_k_dict = {}, {}, {}, {}
        t_dict = dict(zip(self.K, self.t))
        c_dict = dict(zip(self.K, self.c))
        a_dict = dict(zip(self.K, self.a))
        gamma_k_dict = dict(zip(self.K, self.gamma_k))
        return [t_dict, c_dict, a_dict, gamma_k_dict]
    
    def __build_dataframe(self):
        M_data = {'K': self.K, 'K_num': self.K_num, 't': self.t, 'c': self.c, 'a': self.a, 'gamma_k': self.gamma_k}
        M_index = pd.MultiIndex.from_tuples(self.K, names = ["sk", "dk"])
        M_df = pd.DataFrame(data = M_data, index = M_index)
        return M_df

| parameters | variables | baseline value |
|:-|:-:|-:|
| wait time disutility weight | $\mu_w$ | $1.0$ |
| bus capacity | $C$ | $80$ (passengers) |
| travel time between adjacent stops | $t_{i,i+1}$ | $1.5$ (minutes) |
| dwell time at stop $i$ | $t^d_i$ | $0.5$ (minutes) |
| travel time elasticity | $e$ | $-0,5$ (minutes) |
| | | |
| total travel time from stop $i$ to stop $j$ on a local service | $t_{ij}$ | $\sum^{j-1}_{l=i}t_{l,l+1}$$+\sum^{j-1}_{l=i+1}t^d_l$ |
| total travel time savings from running express between stops $i$ and $j$ | $c_{ij}$ | $\sum^{j-1}_{l=i+1}t^d_l$ |
| rate of increase in limited-stop service demand per minute of travel time reduction for O-D pair $k=(s^k,d^k)$ | $a^k$ | $\frac{-e}{t_{s^k,d^k}}$ |
| | | |
| number of passengers | $P$ | $3151$ |
| number of bus trips | $B$ | $24$ |
| number of stops | $S$ | $35$ |
| peak period | $T$ | $120$ (minutes) |

In [239]:
# instance = Generator(35, 120, 3151, 24, 1.0, 80, 1.5, 0.5, -0.5)

# ------------------------------------------------------------------

# hint by Professor Rossi
# (L'Aquila --> Lanciano) simulation instance

#         stops_num    ->    S = 3, (L'Aquila --> intermediate stop --> Lanciano)
#       peak_period    ->    T = all day, 1440 minutes
#    passengers_num    ->    P = 50 passengers
#         bus_trips    ->    B = 2 or 3 bus trips
#     tt_disutility    ->    1.0 (default)
#      bus_capacity    ->    50 passengers
#      tt_adj_stops    ->    1.5 (default) 
# dwell_time_stop_i    ->    0.5 (default) 
#     tt_elasticity    ->   -0.5 (default)

instance = Generator(3, 1440, 50, 3, 1.0, 50, 1.5, 0.5, -0.5)
instance.M_dict

K  K_num    t    c         a                   gamma_k
sk dk                                                             
1  2   (1, 2)      0  1.5  0.0  0.333333                  [(1, 2)]
   3   (1, 3)      1  3.5  0.5  0.142857  [(1, 2), (1, 3), (2, 3)]
2  3   (2, 3)      2  1.5  0.0  0.333333                  [(2, 3)]

In [240]:
print("========== PARAMS ==========")
print()
print(instance.S, "\t[num of stops]")
print(instance.T, "\t[peak period (in minutes)]")
print(instance.P, "\t[num. of passengers]")
print(instance.B, "\t[num. of bus trips]")
print()
print(instance.m, "\t[travel time disutility weight]")
print(instance.C, "\t[bus capacity]")
print(instance.ta, "\t[travel time between adjacent stops (i to j)]")
print(instance.d, "\t[dwell time at stop i]")
print(instance.e, "\t[travel time elasticity]")
print()
print("========== t MATRIX ==========")
"""
    1 --- 2                (dwell time = 0.0, travel time = 1.5)
    1 --- 2 --- 3          (dwell time = 0.5, travel time = 1.5 * 2)
    ...
    9 --- 10               (dwell time = 0.0, travel time = 1.5)
    10 --- NO              (last row -> deleted)
"""
print()
print(instance.t_dict[(1,2)])
print(instance.t_dict[(1,3)])
print(instance.t_dict[(2,3)])
print()
print("(1, 2)", instance.M_dict.loc[1,2].t)
print("(1, 3)", instance.M_dict.loc[1,3].t)
print("(2, 3)", instance.M_dict.loc[2,3].t)
print()
print("========== c MATRIX ==========")
"""
    1 --- 2                (dwell time = 0.0)
    1 --- 2 --- 3          (dwell time = 0.5)
    ...
    9 --- 10               (dwell time = 0.0)
    10 --- NO              (last row -> deleted)
"""
print()
print(instance.c_dict[(1,2)])
print(instance.c_dict[(1,3)])
print(instance.c_dict[(2,3)])
print()
print("(1, 2)", instance.M_dict.loc[1,2].c)
print("(1, 3)", instance.M_dict.loc[1,3].c)
print("(2, 3)", instance.M_dict.loc[2,3].c)
print()
print("========== a MATRIX ==========")
"""
    1 --- 2                (-elasticity / travel time = 1.5)
    1 --- 2 --- 3          (-elasticity / travel time = 1.5 * 2)
    ...
    9 --- 10               (-elasticity / travel time = 1.5)
    10 --- NO              (last row -> deleted)
"""
print()
print(instance.a_dict[(1,2)])
print(instance.a_dict[(1,3)])
print(instance.a_dict[(2,3)])
print()
print("(1, 2)", instance.M_dict.loc[1,2].a)
print("(1, 3)", instance.M_dict.loc[1,3].a)
print("(2, 3)", instance.M_dict.loc[2,3].a)


========== PARAMS ==========

3 	[num of stops]
1440 	[peak period (in minutes)]
50 	[num. of passengers]
3 	[num. of bus trips]

1.0 	[travel time disutility weight]
50 	[bus capacity]
1.5 	[travel time between adjacent stops (i to j)]
0.5 	[dwell time at stop i]
-0.5 	[travel time elasticity]

========== t MATRIX ==========

1.5
3.5
1.5

(1, 2) 1.5
(1, 3) 3.5
(2, 3) 1.5

========== c MATRIX ==========

0.0
0.5
0.0

(1, 2) 0.0
(1, 3) 0.5
(2, 3) 0.0

========== a MATRIX ==========

0.3333333333333333
0.14285714285714285
0.3333333333333333

(1, 2) 0.3333333333333333
(1, 3) 0.14285714285714285
(2, 3) 0.3333333333333333


In [241]:
incr_stop_skip = gb.Model()
incr_stop_skip_lp = "./lp/incr_stop_skip.lp"

if os.path.exists(incr_stop_skip_lp):
    open(incr_stop_skip_lp, "w").close()

In [242]:
""" limited-stop service decision variables """

""" ========== BETA ========== """
# lb = 0.0 (default)
# up = 1.0
# integrality of beta and gamma can be relaxed (proposition 1) 
# vtype = gb.GRB.BINARY -> gb.GRB.CONTINUOUS (Proposition 1)
Beta = incr_stop_skip.addVars(instance.s, ub = 1.0, \
                              vtype = gb.GRB.CONTINUOUS, name = 'beta')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

""" ========== GAMMA ========== """
# lb = 0.0 (default)
# up = 1.0
# integrality of beta and gamma can be relaxed (proposition 1) 
# vtype = gb.GRB.BINARY -> gb.GRB.CONTINUOUS (Proposition 1)
Gamma = incr_stop_skip.addVars(instance.K, ub = 1.0, \
                               vtype = gb.GRB.CONTINUOUS, name = 'gamma')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

In [243]:
""" passengers decision variables """

""" ========== X ========== """
# lb = 0.0 (default)
# up = 1.0
# vtype = GRB.BINARY -> GRB.CONTINUOUS
X = incr_stop_skip.addVars(instance.K, ub = 1.0, \
                           vtype = gb.GRB.CONTINUOUS, name = 'x')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

""" ========== Y ========== """
# lb = 0.0 (default)
# up = 1.0
# vtype = GRB.BINARY -> GRB.CONTINUOUS
Y = incr_stop_skip.addVars(instance.K, ub = 1.0, \
                           vtype = gb.GRB.CONTINUOUS, name = 'y')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

""" ========== Z ========== """
# lb = 0.0 (default)
# up = 1.0
# vtype = GRB.BINARY -> GRB.CONTINUOUS
Z = incr_stop_skip.addVars(instance.K, ub = 1.0, \
                           vtype = gb.GRB.CONTINUOUS, name = 'z')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

""" ========== W ========== """
# lb = 0.0 (default)
# up = constraint (18)
# vtype = GRB.INTEGER
W = incr_stop_skip.addVars(instance.s[:-1], \
                           vtype = gb.GRB.CONTINUOUS, name = 'w')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

In [244]:
""" amount of passengers (assumption 1) """

pk = dict(zip(instance.K, [random.randint(1,instance.P) for k in instance.K]))
print(pk)

""" frequency F constant (assumption 2) """
F = random.randint(1, instance.B-1)
print(F)

{(1, 2): 29, (1, 3): 4, (2, 3): 24}
2


$$\max\sum_{k\in K}p^k\sum_{(i,j)\in\Gamma^k}c_{ij}x^k_{ij}\\-\mu_w\sum_{k\in K}p^k(1-\gamma^k)\frac{1}{2}(\frac{T}{f_0-f}-\frac{T}{f_0})\\-\mu_w\sum_{k\in K}p^k z^k\frac{1}{2}(\frac{T}{f}-\frac{T}{f_0})\tag{1}$$

where:
- $\Gamma^k$ denotes a **set of segments** that can be used to serve O-D pair $k\in K$
- mathematically, for an **O-D pair** $k=(s^k,d^k)\in K$, $\Gamma^k$ is given by $\{(i,j)|i,j\in S,s^k\leq i<j\leq d^k\}$

In [245]:
""" (1) expression - w/ M_dict dataframe """

# objexpr = gb.quicksum(pk[i] for i in instance.M_dict.loc[:].K)\
# * gb.quicksum(instance.M_dict.loc[i].c * X[i] \
#             for i in list(k for j in instance.M_dict.loc[:].gamma_k.values.flatten() for k in j))\
# -(instance.m * (gb.quicksum(pk[i] * (1 - Gamma[i]) \
#                             * 1/2 * (instance.T/(instance.B - F) - instance.T/instance.B) \
#                             for i in instance.M_dict.loc[:].K)))\
# -(instance.m * (gb.quicksum(pk[i] * Z[i] * 1/2 * (instance.T/F - instance.T/instance.B) \
#                             for i in instance.M_dict.loc[:].K)))

""" (1) expression - w/ shortcut dicts """

objexpr = gb.quicksum(pk[i] for i in instance.K) \
* gb.quicksum(instance.c_dict[g] * X[g] for i in instance.K for g in instance.gamma_k_dict[i]) \
- instance.m * gb.quicksum(pk[i] * (1 - Gamma[i]) \
                           * 1/2 * (instance.T/(instance.B - F) - instance.T/instance.B) \
                           for i in instance.K) \
- instance.m * gb.quicksum(pk[i] * Z[i] \
                           * 1/2 * (instance.T/F - instance.T/instance.B) \
                           for i in instance.K)
""" (1) """

incr_stop_skip.setObjective(objexpr, gb.GRB.MAXIMIZE)

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\alpha_{ij}\in\{0,1\}, \forall(i,j)\in\{(i,j)|i,j\in S,i<j\}\\\alpha_{s^+,i},\alpha_{i,s^-}\in\{0,1\}, \forall i\in S\tag{5}$$

In [246]:
""" limited-stop service route constraints (Alpha) """

""" (5) """

K_Alpha = instance.K + [(0, i) for i in instance.s] + [(i, instance.S+1) for i in instance.s]

Alpha = incr_stop_skip.addVars(K_Alpha, vtype = gb.GRB.BINARY, name = 'alpha')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\sum_{i\in S\setminus\{|S|\}}\alpha_{s^+,i}=1\tag{2}$$

In [247]:
""" limited-stop service route constraints (Alpha) """

""" (2) """

expr_2 = gb.quicksum(Alpha[0,i] for i in instance.s[:-1]) == 1

incr_stop_skip.addConstr(expr_2, name = '(2)')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\sum_{i\in S:j<i}\alpha_{ji}+\alpha_{s^+,i}=\sum_{j\in S:j>i}\alpha_{ij}+\alpha_{i,s^-}, \forall i\in S\tag{3}$$

In [248]:
""" limited-stop service route constraints (Alpha) """

""" (3) """

for i in instance.s:
    incr_stop_skip.addConstr(gb.quicksum(Alpha[j,i] for j in instance.s if(j<i)) + \
                             Alpha[0,i] == \
                             gb.quicksum(Alpha[i,j] for j in instance.s if(j>i)) + \
                             Alpha[i, instance.S+1],
                             name = '(3)[' + str(i) + ']')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\sum_{i\in S\setminus\{1\}}\alpha_{i,s^-}=1\tag{4}$$

In [249]:
""" limited-stop service route constraints (Alpha) """

""" (4) """

expr_4 = gb.quicksum(Alpha[i,instance.S+1] for i in instance.s[1:]) == 1

incr_stop_skip.addConstr(expr_4, name = '(4)')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\beta_i=\sum_{j\in S:j>i}\alpha_{ij}+\alpha_{i,s^-}, \forall i\in S\tag{6}$$

In [250]:
""" limited-stop service route constraints (Beta, Gamma) """

""" (6) """

for i in instance.s:
    incr_stop_skip.addConstr(Beta[i] == gb.quicksum(Alpha[i,j] for j in instance.s if(j>i)) + \
                             Alpha[i, instance.S+1], \
                             name = '(6)[' + str(i) + ']')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\gamma^k\leq\beta_{s^k}, \forall k=(s^k,d^k)\in K\tag{7}$$

In [251]:
""" limited-stop service route constraints (Beta, Gamma) """

""" (7) """

expr_7 = (Gamma[(i,j)] <= Beta[i] for i,j in instance.K)

incr_stop_skip.addConstrs(expr_7, name = '(7)')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\gamma^k\leq\beta_{d^k}, \forall k=(s^k,d^k)\in K\tag{8}$$

In [252]:
""" limited-stop service route constraints (Beta, Gamma) """

""" (8) """

expr_8 = (Gamma[(i,j)] <= Beta[j] for i,j in instance.K)

incr_stop_skip.addConstrs(expr_8, name = '(8)')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

$$\beta_i\in\{0,1\}, \forall i\in S\tag{9}$$
$$\gamma^k\in\{0,1\}, \forall k\in K\tag{10}$$

In [253]:
""" limited-stop service route constraints (Beta, Gamma) """

""" (9)(10) """

""" already satisfied and relaxed """

' already satisfied and relaxed '

$$x^k_{ij}\leq\alpha_{ij}, \forall k\in K, (i,j)\in\Gamma^k\tag{11}$$

In [254]:
""" passengers flow constraints """

""" (11) """

incr_stop_skip.addConstrs((X[i,j] <= Alpha[i,j] for i,j in instance.K), \
                          name = '(11)')

incr_stop_skip.update()
incr_stop_skip.write(incr_stop_skip_lp)

In [255]:
""" optimization hook """

incr_stop_skip.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 17 rows, 26 columns and 43 nonzeros
Model fingerprint: 0xf6c7872a
Variable types: 17 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 17 rows and 26 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 16 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%


In [256]:
""" optimization results """
incr_stop_skip.ObjVal, incr_stop_skip.Runtime

(-0.0, 0.027783870697021484)